In [ ]:
import xml.etree.cElementTree as ET
import nltk
from functools import reduce
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import seaborn as sns
from scipy.stats import norm
import numpy
import math
from matplotlib.ticker import ScalarFormatter 
import random
#import word2vec
import os
import pickle

1)

In [ ]:
def tokenize(text, remove_stopwords=True, allowed_words = [], lemat=False):
    stopWords = set(stopwords.words('english'))|set(["''",",''",".''",".,'","...",".,",'``','--'])
    tokens = nltk.wordpunct_tokenize(text)
    tokens = [word.lower() for word in tokens if len(word) > 1]
    tokens = [word for word in tokens if (not (word in stopWords and remove_stopwords) or word in allowed_words)
              and not word.isdigit()]
    
    if lemat:
        porter_stemmer = PorterStemmer()
        tokens = [porter_stemmer.stem(word) for word in tokens]
    return tokens

In [ ]:
root = ET.ElementTree(file='ap/ap.xml').getroot()

news_dic = {}
full_text = []
for node in root:
    docno, text = node
    news_dic[docno.text.strip()] = text.text

full_text = ''.join([v for k,v in news_dic.items()])

In [ ]:
tokens = tokenize(full_text)
fd = nltk.FreqDist(tokens)
most_common_s = set(sorted(fd, key=lambda x: fd[x],reverse=True)[:500])
most_common_s

In [ ]:
window_size = 5
bgm    = nltk.collocations.BigramAssocMeasures()
finder = nltk.collocations.BigramCollocationFinder.from_words(tokens, window_size)
finder.apply_ngram_filter(lambda w1, w2: w1 not in most_common_s or w2 not in most_common_s)
scored = finder.score_ngrams( bgm.pmi  )
scored

2)

In [ ]:
orig_species_f = open('ap/pg1228.txt', 'r')
inputfile = orig_species_f.read()
tokens_orig = tokenize(inputfile, False, allowed_words = ['for'])

In [ ]:
#nltk.draw.dispersion.dispersion_plot(tokens_orig,['plant','instinct','for'])

In [ ]:
def get_apps_for_word_in_text(aWord, aText):
    return [i for i,word in enumerate(aText) if word == '{}'.format(aWord)]

instinct_app = get_apps_for_word_in_text('instinct', tokens_orig)
plant_app = get_apps_for_word_in_text('plant', tokens_orig)
for_app = get_apps_for_word_in_text('for', tokens_orig)

total_words = len(tokens_orig)

Para que termine usamos un salto cada 10 palabras.
Lo siguiente lo corrimos y guardamos en el dir "ap/guassian_sum_calc/*.out".

In [ ]:
## rowo is an acronym of "rate of word occurrency"
rowo_instinct = []
rowo_plant = []
rowo_for = []

In [ ]:
## Read files
for _rowo,_word in [(rowo_instinct,'instict'),(rowo_plant,'plant'),(rowo_for,'for')]:
    with open('ap/gaussian_sum_calc/{}.out'.format(_word),'rb') as fp:
        _rowo.extend(pickle.load(fp))

In [ ]:
sns.plt.subplot(311)
sns.plt.plot(rowo_instinct)
sns.plt.subplot(312)
sns.plt.plot(rowo_plant)
sns.plt.subplot(313)
sns.plt.plot(rowo_for)
sns.plt.show()

In [ ]:
def autocorrelacion(_row):
    pwt_avg = sum([ _row[pos] for pos in range(len(_row)) ]) / len(_row)
    cociente = pwt_avg**2 + sum([ _row[pos]**2 for pos in range(len(_row)) ]) / len(_row)

    taus = []

    for tau in range(1000):
        pwt_tau_avg = sum([ _row[pos]*_row[pos+tau] for pos in range(0,len(_row)-tau)]) / (len(_row)-tau)
        taus.append(pwt_tau_avg - pwt_avg * (sum([_row[pos+tau] for pos in range(0,len(_row)-tau)]) / (len(_row)-tau)))
        taus[tau] /= cociente
    return taus

In [ ]:
auto_rowo_instinct = autocorrelacion(rowo_instinct)
sns.plt.subplot(311)
sns.plt.plot(auto_rowo_instinct)

auto_rowo_plant = autocorrelacion(rowo_plant)
sns.plt.subplot(312)
sns.plt.plot(auto_rowo_plant)

auto_rowo_for = autocorrelacion(rowo_for)
sns.plt.subplot(313)
sns.plt.plot(rowo_for)

sns.plt.show()

In [ ]:
fd_orig = nltk.FreqDist(tokens_orig)
salto = len(fd_orig)//1000
elegidas = dict()
fd_orig_s = sorted(fd_orig, key=lambda x: fd_orig[x],reverse=True)[:100]
fd_orig_s
for e in fd_orig_s:
        elegidas[e] = fd_orig[e]

elegidas

In [ ]:
def freq_in_partition(w,p,p_total,text):
    desde = p*(len(text)//p_total)
    hasta = (p+1)*(len(text)//p_total)
    tot = 0
    for wt in text[desde:hasta]:
        if w==wt:
            tot += 1
    return tot

particiones = 64
entropias_orig = dict()
for w,freq in elegidas.items():
    entropia_w = 0
    for p in range(particiones):
        prob = freq_in_partition(w,p,particiones,tokens_orig)/freq
        if prob != 0:
            entropia_w += prob*(math.log(prob,2))
    entropia_w = -1*entropia_w
    entropias_orig[w] = entropia_w


tokens_shuffled = tokens_orig.copy()
random.shuffle(tokens_shuffled)
entropias_random = dict()
for w,freq in elegidas.items():
    entropia_w = 0
    for p in range(particiones):
        prob = freq_in_partition(w,p,particiones,tokens_shuffled)/freq
        if prob != 0:
            entropia_w += prob*(math.log(prob,2))
    entropia_w = -1*entropia_w
    entropias_random[w] = entropia_w

entropias_orig
entropias_random

In [ ]:
frequencies = []
entropies = []
entropies_shuff = []
for w,freq in elegidas.items():
    frequencies.append(freq)
    entropies.append(entropias_orig[w])
    entropies_shuff.append(entropias_random[w])

h = sns.pointplot(x=frequencies,y=entropies, join = False, ci = None)
g = sns.pointplot(x=frequencies,y=entropies_shuff, join = False, ci = None, color="red")
sns.mpl.rc("figure", figsize=(15,10))
sns.plt.show()

In [ ]:
from gensim import corpora, models, similarities
dictionary = corpora.Dictionary(news_dic.values())
lsi = models.LsiModel([dictionary.doc2bow(text) for text in news_dic.values()], id2word=dictionary, num_topics=4)

In [ ]:
def to_lsi(word):
    vec_bow = dictionary.doc2bow(word.lower())
    return lsi[vec_bow] # convert the query to LSI space 
lsi.print_topics(5)

In [ ]:
model = models.Word2Vec(news_dic.values(), size=15, window=20, min_count=5, workers=4)

In [ ]:
word_sim_f = open('wordsim353_sim_rel/wordsim_similarity_goldstandard.txt')
error = 0
for line in word_sim_f:
    w1, w2, v2 = line.split()
    if w1 in model.wv.vocab and w2 in model.wv.vocab:
        v2 = float(v2)
        v1 = model.wv.similarity(w1, w2)*10
        epsilon = (v1-v2)**2
        error += epsilon
    print(w1,w2,v1,v2,epsilon)

error